In [1]:
%matplotlib inline

In [2]:
import LFPy
import lfpykit
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# LFPy.Cell parameters
cellParameters = {
    'morphology': 'L5_Mainen96_LFPy.hoc',  # morphology file
    'v_init': -65,                         # initial voltage
    'cm': 1.0,                             # membrane capacitance
    'Ra': 150,                             # axial resistivity
    'passive': True,                       # insert passive channels
    'passive_parameters': {"g_pas": 1. / 3E4,
                           "e_pas": -65},  # passive params
    'dt': 2**-4,                           # simulation time res
    'nsegs_method': 'lambda_f',    # discretization rule
    'lambda_f': 1000               # frequency (Hz)
}

# create LFPy.Cell instance
cell = LFPy.Cell(**cellParameters)
cell.set_rotation(x=4.98919, y=-4.33261, z=0.)

print(f'total number of segments: {cell.totnsegs}')

# parameters for line source potential
el_params = dict(
    x = np.linspace(0, 1000, 1001),
    y = np.zeros(1001),
    z = np.zeros(1001),
    sigma = 0.3
)

cell.simulate(rec_imem=True)

total number of segments: 3078


In [4]:
# create line-source potential predictor
lsp = lfpykit.LineSourcePotential(cell, **el_params)

In [5]:
%%prun -s cumulative -q -l 50 -T prun0
for i in range(100):
    lsp.get_transformation_matrix()

OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


 
*** Profile printout saved to text file 'prun0'.


In [6]:
print(open('prun0', 'r').read())

         2670927 function calls (2651191 primitive calls) in 4.223 seconds

   Ordered by: cumulative time
   List reduced from 1402 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     70/1    0.000    0.000    4.250    4.250 {built-in method builtins.exec}
      100    0.004    0.000    3.933    0.039 models.py:441(get_transformation_matrix)
      100    2.328    0.023    2.332    0.023 models.py:463(_get_transform)
      100    0.001    0.000    1.304    0.013 dispatcher.py:388(_compile_for_args)
      100    0.002    0.000    1.283    0.013 dispatcher.py:915(compile)
      100    0.000    0.000    1.269    0.013 caching.py:639(load_overload)
61409/61309    0.865    0.000    1.258    0.000 ffi.py:149(__call__)
      100    0.000    0.000    1.167    0.012 caching.py:650(_load_overload)
      100    0.000    0.000    1.116    0.011 caching.py:404(rebuild)
      100    0.001    0.000    1.116    0.011 compiler.py:210(_rebuild)
   

In [7]:
# create point-source potential predictor
psp = lfpykit.PointSourcePotential(cell, **el_params)

In [8]:
%%prun -s cumulative -q -l 20 -T prun1
for i in range(100):
    psp.get_transformation_matrix()

 
*** Profile printout saved to text file 'prun1'.


In [9]:
print(open('prun1', 'r').read())

         3203403 function calls in 8.508 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    8.508    8.508 {built-in method builtins.exec}
        1    0.010    0.010    8.508    8.508 <string>:1(<module>)
      100    0.232    0.002    8.498    0.085 models.py:279(get_transformation_matrix)
   100100    1.279    0.000    8.266    0.000 lfpcalc.py:604(calc_lfp_pointsource)
   300300    0.065    0.000    6.647    0.000 {method 'mean' of 'numpy.ndarray' objects}
   300300    0.806    0.000    6.582    0.000 _methods.py:162(_mean)
   300300    5.484    0.000    5.484    0.000 {method 'reduce' of 'numpy.ufunc' objects}
   100100    0.340    0.000    0.340    0.000 lfpcalc.py:686(_check_rlimit_point)
   300300    0.210    0.000    0.245    0.000 _methods.py:66(_count_reduce_items)
   600600    0.026    0.000    0.026    0.000 {built-in method builtins.issubclass}
   600600    0.026    0.000    0.026